# Deep learning for computer vision

Научимся тренировать сверточные нейронные сети для распознования изображений.

# Tiny ImageNet dataset
Будем практиковаться с Tiny Image Net датасетом
* 100k изображений размера 3x64x64
* 200 разных классов: змеи, пауки, кошки, грузовики, и т.д.

In [ ]:
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
import time
from tqdm import tqdm

# если у вас больше 1й видеокарты, полезно бывает ограничить видимость
# до 1й в случае если не собираетесь распараллеливать обучение
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '#номер карты'

In [ ]:
import tiny_imagenet
tiny_imagenet.download(".")

In [ ]:
dataset_root = "tiny-imagenet-200"

train_dataset = torchvision.datasets.ImageFolder(
    os.path.join(dataset_root, "train"),
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2768, 0.2689, 0.2819])
    ]))

val_dataset = torchvision.datasets.ImageFolder(
    os.path.join(dataset_root, "val"),
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2768, 0.2689, 0.2819])
    ]))

batch_size = 128
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=1)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=1)

## Image examples ##



<tr>
    <td> <img src="tinyim3.png" alt="Drawing" style="width:90%"/> </td>
    <td> <img src="tinyim2.png" alt="Drawing" style="width:90%"/> </td>
</tr>


<tr>
    <td> <img src="tiniim.png" alt="Drawing" style="width:90%"/> </td>
</tr>

# Building a network

Простые нейронные сети с наслоением стандартных слоев могут быть реализованы с помощью `torch.nn.Sequential`

In [ ]:
# a special module that converts [batch, channel, w, h] to [batch, units]
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

Для начала начнем с полносвязной сетью

In [ ]:
model = nn.Sequential()
model.add_module('flatten', Flatten())
model.add_module('dense1', nn.Linear(3 * 64 * 64, 512))
model.add_module('dense1_relu', nn.ReLU())
model.add_module('dense2', nn.Linear(512, 256))
model.add_module('dense2_relu', nn.ReLU())
model.add_module('dense3', nn.Linear(256, 128))
model.add_module('dense3_relu', nn.ReLU())
model.add_module('dropout', nn.Dropout(0.05))
model.add_module('dense4', nn.Linear(128, 64))
model.add_module('dense3_relu', nn.ReLU())
model.add_module('logits', nn.Linear(64, 200)) # logits на 200 классов

# эквивалентно, но слои без имен
# model == nn.Sequential(
#     Flatten(),
#     nn.Linear(3 * 64 * 64, 1064),
#     nn.Linear(3 * 64 * 64, 1064),
#     ...
# )

### Напишем тренировочную и тестовую "рутину"

Ниже приводится пример! того как можно разбить на различные функции процесс обучения и валидации, вместо того чтобы писать один большой громоздкий цикл, как в предыдущих семинарах.

In [ ]:
def training_step(batch, model, device=torch.device('cpu')):
    data, target = batch
    
    # закидываем данные и модель на один и тот же device
    data, target = data.to(device), target.to(device)
    model.to(device)
    
    # включаем train mode (обязательно например при наличии dropout, batch_norm и т.д.)
    model.train()
    
    logits = model(data)
    loss = F.cross_entropy(logits, target)
    
    # loss.item() эквивалентно loss.detach().cpu().numpy()
    logs = {'train_ce_loss': loss.item()}
    
    return {'loss': loss, 'log': logs}

def train_one_epoch(model, loader, optimizer, scheduler=None, device=torch.device('cpu')):
    logs = []
    for batch in tqdm(loader):
        output = training_step(batch, model, device)
        loss = output['loss']
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if scheduler is not None:
            scheduler.step()
        logs.append(output['log'])
    return logs

def train_epoch_end(logs):
    avg_train_ce_loss = np.mean([x['train_ce_loss'] for x in logs])
    return {'avg_train_ce_loss': avg_train_ce_loss}

def validation_step(batch, model, device=torch.device('cpu')):
    data, target = batch
    
    # закидываем данные и модель на один и тот же device
    data, target = data.to(device), target.to(device)
    model.to(device)
    
    # включаем eval mode (обязательно например при наличии dropout, batch_norm и т.д.)
    model.eval()
    
    start_time = time.perf_counter()
    with torch.no_grad():
        logits = model(data)
    inference_time_per_batch = time.perf_counter() - start_time
    
    pred = logits.argmax(dim=1, keepdim=True)
    
    correct = pred.eq(target.view_as(pred)).sum().item()
    log = {'correct': correct, 'inference_time_per_batch': inference_time_per_batch, 'amount': pred.shape[0]}
    return {'log': log}

def val_one_epoch(model, loader, device=torch.device('cpu')):
    logs = []
    for batch in tqdm(loader):
        output = validation_step(batch, model, device)
        logs.append(output['log'])
    return logs
        
def validation_epoch_end(logs):
    total_amount = np.sum([x['amount'] for x in logs])
    total_correct = np.sum([x['correct'] for x in logs])
    accuracy = 100 * total_correct / total_amount 
    avg_inference_time_per_batch = np.mean([x['inference_time_per_batch'] for x in logs])
    return {'val_accuracy (%)': accuracy, 'avg_inference_time_per_batch (sec)': avg_inference_time_per_batch}
    
def configure_optimizers(model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
    scheduler = None
    return optimizer, scheduler

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer, scheduler = configure_optimizers(model)
num_epochs = 3

for epoch in range(num_epochs):
    print(f"Epoch {epoch}:")
    train_logs = train_one_epoch(model, train_loader, optimizer, scheduler, device)
    print(train_epoch_end(train_logs))
    val_logs = val_one_epoch(model, val_loader, device)
    print(validation_epoch_end(val_logs))

Не ждите все 100 эпох. Если видите, что точность на валидации не растет в течение 5-20 эпох - можете остановить процесс.
```

```

```

```

```

```

```

```

```

```

### Final test

In [ ]:
print("Test stage:")
val_logs = val_one_epoch(model, val_loader)
print(validation_epoch_end(val_logs))

## Task I: small convolution net
### Первый шаг

Давайте создадим маленькую сверточную сеть с примерно следующей архитектурой:
* Input layer
* 3x3 convolution with 128 filters and _ReLU_ activation
* 2x2 pooling (or set previous convolution stride to 2)
* Flatten
* Dense layer with 1024 neurons and _ReLU_ activation
* 30% dropout
* Output dense layer.


__Convolutional layers__ отдельный класс слоев в торче со своим набором параметров

__`...`__

__`model.add_module('conv1', nn.Conv2d(in_channels=3, out_channels=128, kernel_size=3)) # convolution`__

__`model.add_module('pool1', nn.MaxPool2d(2)) # max pooling 2x2`__

__`...`__


In [ ]:
# your code here

Теперь обучим:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer, scheduler = configure_optimizers(model)
num_epochs = 3

for epoch in range(num_epochs):
    print(f"Epoch {epoch}:")
    train_logs = train_one_epoch(model, train_loader, optimizer, scheduler, device)
    print(train_epoch_end(train_logs))
    val_logs = val_one_epoch(model, val_loader, device)
    print(validation_epoch_end(val_logs))

```

```

```

```

```

```

```

```

```

```

__Hint:__ Если не хотите вручную считать размеры линейного слоя, можете сначала указать любой размер и запустить. Вы увидите нечто похожее:

__`RuntimeError: size mismatch, m1: [5 x 1960], m2: [1 x 64] at /some/long/path/to/torch/operation`__

Видите число __1960__? Это как раз нужный вам размер входа линейного слоя.

## Task 2: adding normalization

* Добавьте batch norm (со стандартными параметрами) между convolution и ReLU
  * nn.BatchNorm*d (1d for dense, 2d for conv)
  * обычно его добавляют после linear/conv но перед нелинейностью

In [ ]:
# your code here

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer, scheduler = configure_optimizers(model)
num_epochs = 3

for epoch in range(num_epochs):
    print(f"Epoch {epoch}:")
    train_logs = train_one_epoch(model, train_loader, optimizer, scheduler, device)
    print(train_epoch_end(train_logs))
    val_logs = val_one_epoch(model, val_loader, device)
    print(validation_epoch_end(val_logs))


```

```

```

```

```

```

```

```

```

```

```

```

```

```
## Task 3: Data Augmentation

** Augmenti - A spell used to produce water from a wand (Harry Potter Wiki) **

<img src="HagridsHut_PM_B6C28_Hagrid_sHutFireHarryFang.jpg" style="width:80%">

В торче есть инструмент полезный для data preprocessing и augmentation.

С помощью него мы определяем следующий пайплайн:
* поворачиваем изображение (augmentation)
* рандомно делает горизонтальный флип (augmentation)
* нормализуем изображение (preprocessing)

Во время тестирования нам не нужны аугментации, а нужно оставить только __нормализацию__.

In [ ]:
import torchvision
from torchvision import transforms

dataset_root = "tiny-imagenet-200"

train_dataset = torchvision.datasets.ImageFolder(
    os.path.join(dataset_root, "train"),
    transform=transforms.Compose([
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2768, 0.2689, 0.2819])
    ]))

val_dataset = torchvision.datasets.ImageFolder(
    os.path.join(dataset_root, "val"),
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2768, 0.2689, 0.2819])
    ]))

batch_size = 128
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=1)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer, scheduler = configure_optimizers(model)
num_epochs = 3

for epoch in range(num_epochs):
    print(f"Epoch {epoch}:")
    train_logs = train_one_epoch(model, train_loader, optimizer, scheduler, device)
    print(train_epoch_end(train_logs))
    val_logs = val_one_epoch(model, val_loader, device)
    print(validation_epoch_end(val_logs))